In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params
import os
import gc

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.70'

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
os.environ['MKL_NUM_THREADS']='1' 
os.environ['OPENBLAS_NUM_THREADS']='1'

os.environ["NUM_INTER_THREADS"]="1"
os.environ["NUM_INTRA_THREADS"]="1"
os.environ.update(
        XLA_FLAGS=(
            '--xla_cpu_multi_thread_eigen=false '
            'intra_op_parallelism_threads=5 '
            'inter_op_parallelism_threads=5 '
            'OMP_NUM_THREADS=1  '
        ),
        XLA_PYTHON_CLIENT_PREALLOCATE='false',
    )
#os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=10")

In [3]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR11234331.assembly.len15.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.pkl"

In [4]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [5]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,VAQHRPLIPVLPERPQYSGRSLHSPAAVSMPLSDLDLLAVTDLSLS...,50
1,1,MERIELRHANSEEGRSQKTSCRYKFF,28
2,2,MQTLKRGGHRRLAVDTNFSEKPED,26
3,3,MPQLNSLHTSVSLSVTPL,20
4,4,MYCLAMLHPTQVRGR,17
...,...,...,...
12753725,12753725,MFFSSPSRRVFSLSNESS,20
12753726,12753726,MNPITPICTVHSTSIAFLFFILAV,26
12753727,12753727,CGLWGGQSVRTQRQLVMSDAQHVPSPAAGSELVSVLWAVGRQLLLL...,163
12753728,12753728,MPRAMRKRHRSRKRES,18


Remove Duplicate

In [6]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,VAQHRPLIPVLPERPQYSGRSLHSPAAVSMPLSDLDLLAVTDLSLS...,50
1,1,MERIELRHANSEEGRSQKTSCRYKFF,28
2,2,MQTLKRGGHRRLAVDTNFSEKPED,26
3,3,MPQLNSLHTSVSLSVTPL,20
4,4,MYCLAMLHPTQVRGR,17
...,...,...,...
12753717,12753717,EPQRRSARLSAKPAPPKAEPKPKKPPAAKKADKAQKRKKGKADSGKDA,48
12753718,12753718,HLFQNLPFLFFFFELCQPSLLLVAFWALALLLEGRV,37
12753720,12753720,PEAKVGGLCSRWKDSVFVRMVLELKVTVRHPRLRIWSLWWIFNSAG...,97
12753721,12753721,MKKVAILQENALLLIN,18


Divide records , this one did not relate to ID

In [15]:
#0-613834
#613834-1277668
#1277668-1841502
#1841502-2455336
#2455336 -3069170
# 3069170-3683004
#3683004-4296838
# 4296838 - 4910674 
df1 = df.iloc[0:200000, :]
df2 = df.iloc[200000:613834, :]


#df1_1 = df.iloc[200000:613834, :]
#df2 = df.iloc[613834:1277668, :]
#df3 = df.iloc[1277668:1841502, :]
#df4 = df.iloc[1841502:2455336, :]
#df5 = df.iloc[2455336:3069170, :]
#df6 = df.iloc[3069170:3683004, :]
#df7 = df.iloc[3683004:4296838, :]
#df8 = df.iloc[4296838:4910674, :]

In [16]:
df2

,ID,Sequence,length
324050,324050,MWITPEFNKQVASKLQLLGK,22
324052,324052,FLLYQQDLVLHVYVMVQQQLLAIQVDPADLLIPVFPLVL,40
324053,324053,QSLLNISNVVKSPISGDLWIVLGSEDHLAVHFSSSLTSLLSCSLS,46
324054,324054,KQVRITIIIFNRQLITQLTQLNETLITEIGVVYYDDICWWLFKSWH...,52
324055,324055,SRWRCRSSRSLLSGRWRSVRSVSSPRTSRPLSVVLLSPPGIMSLLA...,469
...,...,...,...
1117797,1117797,RNRNLFWKLNPFWLKLRGPNDRTLDQLDCFISSHPEPQNINLLWDT...,88
1117798,1117798,SSNLVMWVADVEHGNKGISDHSTIILMLRFGQRNRN,36
1117799,1117799,HLDFTLPKEDDKSSATQSPAENPERKPPGGKKERRNTEVNEQKKET...,165
1117802,1117802,KDVAPRTFALIKLLLSLRLYFTMATKSRALEAKTRMMTRK,40


In [9]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [ ]:
createREPs(df1,"/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.0_200000.pkl")

In [ ]:
createREPs(df2,"/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.200000_613834.pkl")

In [15]:
_h_avg, h_final, c_final= get_reps(df1['Sequence'].to_list())
df1['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
df1.to_pickle("/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.0_200000.pkl") # dont forget to change the file name to 0_613834

KeyboardInterrupt: 

----- base ---

In [ ]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

In [ ]:
df.to_pickle(plk)
